In [10]:
from requests.auth import HTTPBasicAuth
from impala.dbapi import connect
from datetime import datetime
from pyhive import hive
import pandas as pd
import requests
import sqlparse
import json
import sys
import os
import re
import numpy as np

In [16]:
'''Variables'''
## Atlas dev
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}
credentials = HTTPBasicAuth('atlasadmin', 'xNB8lZ!Dut')
atlas_urls = ["172.30.213.141","172.30.213.142"]
atlas_port = 31000

## Hive dev
hive_host = 'edh-master-01d.root.corp'
hive_port = 10000
hive_user = 'admin'


## Impala dev
impala_host = '172.30.213.211'
impala_port = 21050
impala_user = 'admin'

In [11]:
#HIVE
def show_create_table( hive_database, table_name ):
    #Creo la conexión y obtengo el cursos
    hive_conn = hive.connect(host=hive_host, port=hive_port, username=hive_user, database=hive_database)
    hive_cursor = hive_conn.cursor()

    # Obtengo el create
    hive_cursor.execute(f'SHOW CREATE TABLE {hive_database}.{table_name}')
    databases = list(map(lambda x: x[0], hive_cursor.fetchall()))
    
    #Lo formateo a código ejecutable
    create_table_query = " ".join(databases)
    return create_table_query

In [12]:
#HIVE
def obtener_comentario_tabla( create_table_completo ):
    create_table_completo = create_table_completo.split('COMMENT')
    for elemento in create_table_completo :
        if 'partitioned by' in elemento.lower():
            # Encuentra la posición de 'PARTITIONED BY'
            posicion_partitioned_by = elemento.find('PARTITIONED BY')

            # Obtiene todo antes de 'PARTITIONED BY'
            comentario_tabla = elemento[:posicion_partitioned_by].strip()

            return comentario_tabla

In [36]:
# Sabemos que el comentario de todos los campos fecha_proceso es 'Fecha de Proceso de archivo'
def procesar_fecha_proceso(fila):
    if fila['campo']=='fecha_proceso':
        return 'Fecha de Proceso de archivo'
    else:
        return fila['comentario']

In [5]:
df = pd.read_csv('data/metadata_hive.csv', index_col=0)

In [6]:
df_1 = df.copy()

In [9]:
# Crear una nueva columna 'registro' para determinar cuándo se debe insertar el comentario de tabla
df_1['registro'] = (df_1['tabla'] != df_1['tabla'].shift()).cumsum()

In [24]:
modified_dfs = []

In [25]:
# Iterar sobre grupos y agregar registros según la condición
for name, group in df_1.groupby('registro'):
    # Obtener comment de tabla
    create_table = show_create_table(group['base'].iloc[0], group['tabla'].iloc[0])
    comentario_tabla = obtener_comentario_tabla(create_table)
    
    # Nueva row con comment
    new_row = pd.DataFrame({'campo': [None], 'tipo_dato': [None], 'comentario': [comentario_tabla],
                            'base': [group['base'].iloc[0]], 'tabla': [group['tabla'].iloc[0]], 'entidad': [None], 'zona': [None]})
    
    # Concatenar el grupo + row comment
    modified_group = pd.concat([group, new_row], ignore_index=True)
    
    # Añadir el grupo modificado a la lista
    modified_dfs.append(modified_group)


In [26]:
# Concatenar todos los DataFrames modificados en uno solo
df_modified = pd.concat(modified_dfs, ignore_index=True)

# Eliminar la columna 'registro' si no la necesitas en el resultado final
df_modified = df_modified.drop(columns=['registro'])

In [39]:
# Aplica la función a cada fila de la columna 'comentario'
df_modified['comentario'] = df_modified.apply(procesar_fecha_proceso, axis=1)

In [1]:
df_modified

NameError: name 'df_modified' is not defined

In [11]:
#Guardamos
df_modified.to_csv('data_procesada/metadata_hive_completo.csv',sep='|', index=False)

NameError: name 'df_modified' is not defined

In [12]:
df = pd.read_csv('data_procesada/metadata_hive_completo.csv', sep='|')

In [9]:
df.loc[ (df['tabla'] == 'productos_aampr') & (df['campo'].isna()) ]

,campo,tipo_dato,comentario,base,tabla,entidad,zona
118,NaN,NaN,'Maestro de productos general',de_ber_2cur,productos_aampr,NaN,NaN


In [14]:
df[df['comentario']=="'Maestro de productos general'"]

,campo,tipo_dato,comentario,base,tabla,entidad,zona
118,NaN,NaN,'Maestro de productos general',de_ber_2cur,productos_aampr,NaN,NaN
